# Prepare ecoinvent data 
(calculate IPCC 2013 results and join them with metadata from the Activity Overview file)

In [38]:
import brightway2 as bw
import pandas as pd
import operator
from itertools import islice
import multiprocessing
import numpy as np

DATA_PATH = "../data/"

# Steps in this notebook:

1. Select brightway project and within that project the ecoinvent 3.4 database (cut-off model)
2. Calculate IPCC 2013 results (GWP100a)
3. Load the Activity Overview file and add the LCA results to it.
4. Save the Activity Overview with LCIA results to Excel

## 1. Select brightway project and within that project the ecoinvent 3.4 database (cut-off model)

#### Project

In [2]:
bw.projects

Brightway2 projects manager with 13 objects:
	C4P
	LCA-PR
	MIN-TEA
	Multifunctionality
	default
	ei33
	ei34
	ei35
	ei36
	ei37
	ei38
	test
	video
Use `projects.report()` to get a report on all projects.

In [3]:
bw.projects.set_current('ei34')

#### Database

In [10]:
eidb = bw.Database('ecoinvent 3.4 cutoff')
db = eidb.load()  # the actual data; we need that for the setting the right values in the functional unit

#### Activities

In [7]:
# activities could be filtered here, but we just take the entire database for now
activities = [act.key for act in eidb]
print(len(activities))

14889


#### Functional Units

In [13]:
# production amount for every activity (normally 1, but there are exceptions, e.g. treatment datasets = -1)
functional_units = []  # list of dicts
for act in activities:
    functional_units.append({act: db[act]['production amount']})

In [14]:
functional_units[:2]

[{('ecoinvent 3.4 cutoff', 'f17ad296e98b487f99c56ec5232f3028'): 1.0},
 {('ecoinvent 3.4 cutoff', '8908fda13881d7c66ffacde9eec9792e'): -1.0}]

#### Define LCIA method

In [28]:
method = ('IPCC 2013 as in EXIOBASE', 'climate change', 'GWP 100a')

In [30]:
bw.methods[method]

{'description': 'Climate change assessment consistent with CFs in Exiobase',
 'filename': 'LCIA_methods_for_import.xlsx',
 'unit': 'kg CO2-Eq',
 'abbreviation': 'ipcc-2013-as-in-exiobasecg.b68cffbf8dda29c989c67ff2c7aaedf9',
 'num_cfs': 61}

In [32]:
# one could go deeper here to see the characterization factors like this
#bw.Method(method).load()

# 2. Calculate IPCC 2013 results (GWP100a)

#### Calculation setup

In [34]:
cs_name = "all activities"
bw.calculation_setups[cs_name] = {
    'inv': functional_units,
    'ia': [method]
}

#### Multi-LCA (takes several minutes)

In [35]:
from time import time
start = time()

MLCA = bw.MultiLCA(cs_name)

print(time() - start)

170.7026059627533


# 3. Load the Activity Overview file and add the LCA results to it.

#### LCIA results dataframe

In [36]:
MLCA.results

array([[7.48451650e-01],
       [1.99555345e+00],
       [2.80866979e+00],
       ...,
       [9.56070389e+04],
       [1.00171445e-01],
       [2.36150720e+00]])

In [75]:
df_LCIA = pd.DataFrame(MLCA.results, columns=[method], index=[str(a) for a in activities]).reset_index()
df_LCIA.rename(columns={'index': 'key'}, inplace=True)
print(df_LCIA.shape)
df_LCIA.head()

(14889, 2)
key                                                      object
(IPCC 2013 as in EXIOBASE, climate change, GWP 100a)    float64
dtype: object


,key,"(IPCC 2013 as in EXIOBASE, climate change, GWP 100a)"
0,"('ecoinvent 3.4 cutoff', 'f17ad296e98b487f99c5...",0.748452
1,"('ecoinvent 3.4 cutoff', '8908fda13881d7c66ffa...",1.995553
2,"('ecoinvent 3.4 cutoff', '33053e7d9e459bc2453d...",2.808670
3,"('ecoinvent 3.4 cutoff', 'd9b9b19e7cae0a447de5...",0.018378
4,"('ecoinvent 3.4 cutoff', 'c260d00631b97a7e3a5e...",692.789888


#### Load the Activity Overview

In [86]:
activity_overview = f"{DATA_PATH}ecoinvent/activity_overview_3.4_cut-off_LCIA_without_results.xlsx"
AO = pd.read_excel(activity_overview)
print(AO.shape)
AO.head(2)

(14889, 35)


,key,#,id,filename,activityName,geography,startDate,endDate,specialActivityType,technologyLevel,...,CPC,amount,unitName,productionVolumeAmount,absolute PV,productionVolumeComment,price,By-product classification,mft,functional unit
0,"('ecoinvent 3.4 cutoff', '0008dcb8309bd0ca4820...",3712,400461dd-45f4-4bcc-8eae-afa68e57664a,400461dd-45f4-4bcc-8eae-afa68e57664a_2966d161-...,market group for heavy fuel oil,RER,2015-01-01,2017-12-31,market group,Current,...,33370: Fuel oils n.e.c.,1,kg,1.019820e+11,True,NaN,0.0000,allocatable product,non-mft,1
1,"('ecoinvent 3.4 cutoff', '001145735c7cee63ae10...",402,0766970d-6c54-497e-b458-2626f56d9e0c,0766970d-6c54-497e-b458-2626f56d9e0c_45fbbc41-...,"treatment of municipal solid waste, incineration",AT,2006-01-01,2017-12-31,ordinary transforming activity,Current,...,17300: Steam and hot water,1,MJ,4.823674e+09,True,NaN,0.0106,allocatable product,non-mft,1


#### Add the LCA results data

In [88]:
#AOr = AO.join(df_LCIA, on='key')
AOr = pd.merge(AO, df_LCIA, on='key')
AOr.head(2)

,key,#,id,filename,activityName,geography,startDate,endDate,specialActivityType,technologyLevel,...,amount,unitName,productionVolumeAmount,absolute PV,productionVolumeComment,price,By-product classification,mft,functional unit,"(IPCC 2013 as in EXIOBASE, climate change, GWP 100a)"
0,"('ecoinvent 3.4 cutoff', '0008dcb8309bd0ca4820...",3712,400461dd-45f4-4bcc-8eae-afa68e57664a,400461dd-45f4-4bcc-8eae-afa68e57664a_2966d161-...,market group for heavy fuel oil,RER,2015-01-01,2017-12-31,market group,Current,...,1,kg,1.019820e+11,True,NaN,0.0000,allocatable product,non-mft,1,0.470319
1,"('ecoinvent 3.4 cutoff', '001145735c7cee63ae10...",402,0766970d-6c54-497e-b458-2626f56d9e0c,0766970d-6c54-497e-b458-2626f56d9e0c_45fbbc41-...,"treatment of municipal solid waste, incineration",AT,2006-01-01,2017-12-31,ordinary transforming activity,Current,...,1,MJ,4.823674e+09,True,NaN,0.0106,allocatable product,non-mft,1,0.000000


# 4. Save the Activity Overview with LCIA results to Excel

In [85]:
AOr.to_excel(f"{DATA_PATH}/ecoinvent/activity_overview_3.4_cut-off_LCIA_results.xlsx", 
                      sheet_name='activity overview')